In [3]:
!pip install -U s3fs

     |████████████████████████████████| 125 kB 2.7 MB/s eta 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.8.1
    Uninstalling fsspec-2021.8.1:
      Successfully uninstalled fsspec-2021.8.1
  Attempting uninstall: s3fs
    Found existing installation: s3fs 2021.8.0
    Uninstalling s3fs-2021.8.0:
      Successfully uninstalled s3fs-2021.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2021.8.1 requires fsspec==2021.08.1, but you have fsspec 2021.10.0 which is incompatible.


In [1]:
from rich import print

# Pangeo Forge: Transforming archival data into analysis-ready, cloud-optimized (ARCO)  formats

**Charles Stern** ([@cisaacstern](http://github.com/cisaacstern)), Data Infrastructure Engineer, Lamont-Doherty Earth Observatory (LDEO)

Presentation Repo: https://github.com/cisaacstern/pangeo-forge-slides

### NOAA Optimum Interpolation Sea Surface Temperature (OISST)

For this example we will consider the [NOAA OISST](https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2.html) dataset. This dataset serves as a generic example of how many source files, each representing a temporal subset of a larger dataset, can be transformed into a single, consolidated ARCO datastore using Pangeo Forge.

## NOAA OISST `http` index
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/
```
 │
 ├──198109/
 │   ├──oisst-avhrr-v02r01.19810901.nc
 │  ...
 │   └──oisst-avhrr-v02r01.19810930.nc
...
 └──202110/
```

In [2]:
!wget 'https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/198109/oisst-avhrr-v02r01.19810901.nc'

--2021-10-13 02:11:35--  https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/198109/oisst-avhrr-v02r01.19810901.nc
Resolving www.ncei.noaa.gov (www.ncei.noaa.gov)... 2610:20:8040:2::178, 2610:20:8040:2::168, 2610:20:8040:2::171, ...
Connecting to www.ncei.noaa.gov (www.ncei.noaa.gov)|2610:20:8040:2::178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1714749 (1.6M) [application/x-netcdf]
Saving to: ‘oisst-avhrr-v02r01.19810901.nc’

oisst-avhrr-v02r01. 100%[===================>]   1.63M  2.02MB/s    in 0.8s    

2021-10-13 02:11:37 (2.02 MB/s) - ‘oisst-avhrr-v02r01.19810901.nc’ saved [1714749/1714749]



In [3]:
import xarray as xr

ds = xr.open_dataset("oisst-avhrr-v02r01.19810901.nc")
nbytes = ds.nbytes
print(f"{nbytes/1e6:.2f} MB")
ds

16.60 MB

<xarray.Dataset>
Dimensions:  (time: 1, zlev: 1, lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1981-09-01T12:00:00
  * zlev     (zlev) float32 0.0
Data variables:
    anom     (time, zlev, lat, lon) float32 ...
    err      (time, zlev, lat, lon) float32 ...
    ice      (time, zlev, lat, lon) float32 ...
    sst      (time, zlev, lat, lon) float32 ...
Attributes: (12/37)
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    id:                         oisst-avhrr-v02r01.19810901.nc
    naming_authority:           gov.noaa.ncei
    summary:                    NOAAs 1/4-degree Daily Optimum Interpolation ...
    cdm_data_type:              Grid
    ...                         ...
    metadata_link:              https://doi.org/10.25921/RE9P-PT57
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    sensor:                     Thermometer, AVHRR
    Conventions:                CF-1.6, ACDD-1.3
    references:                 Reynolds, et al.(2007) Daily High-Resolution-...

In [4]:
import pandas as pd

# Our target range includes 14372 files
dates = pd.date_range("1981-09-01", "2021-01-05", freq="D")

print(f"{len(dates)} files -> {(len(dates)*nbytes)/1e9:.2f} GB")

14372 files -> 238.54 GB

### The old way: start downloading

This will take awhile! 

And the end result is likely not well-situated for parallel computation.

# A better way: Pangeo Forge

`pangeo-forge-recipes` provides logic for transforming all of these source files into a single consolidated zarr store.

## What's a `recipe`?

A `recipe` is a Python file which can "see" all of the source files, and also knows how to logically arrange them into a cohesive dataset.

<img src='architecture.png'>

In [5]:
# https://github.com/pangeo-forge/staged-recipes/blob/master/recipes/noaa-oisst/recipe.py

from noaa_oisst_recipe import recipe

for i, kv in enumerate(recipe.file_pattern.items()):
    if i < 2 or i > 14369:
        print(kv[0], kv[1])

(DimIndex(name='time', index=0, sequence_len=14372, operation=<CombineOp.CONCAT: 2>),)
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhr
r/198109/oisst-avhrr-v02r01.19810901.nc

(DimIndex(name='time', index=1, sequence_len=14372, operation=<CombineOp.CONCAT: 2>),)
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhr
r/198109/oisst-avhrr-v02r01.19810902.nc

(DimIndex(name='time', index=14370, sequence_len=14372, operation=<CombineOp.CONCAT: 2>),)
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhr
r/202101/oisst-avhrr-v02r01.20210104.nc

(DimIndex(name='time', index=14371, sequence_len=14372, operation=<CombineOp.CONCAT: 2>),)
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhr
r/202101/oisst-avhrr-v02r01.20210105.nc

# Zarr format

- Compressed, N-dimensional arrays of any NumPy data type
- Chunking in any dimension
- Read & write array chunks concurrently from multiple threads or processes

https://zarr.readthedocs.io/en/stable/

# Zarr build
1. Cache files to cloud
2. Write Zarr "skeleton" to target location
3. Write Zarr chunks
4. Consolidate metadata

> For parallel computation with Dask, 50-100 MB chunks tend to work well. In Pangeo Forge, contributors specify chunking with the `XarrayZarrRecipe.target_chunks` keyword; e.g., `target_chunks={"time": 10}`.

In [6]:
import s3fs
import xarray as xr

endpoint_url = "https://ncsa.osn.xsede.org"
fs_osn = s3fs.S3FileSystem(anon=True, client_kwargs={"endpoint_url": endpoint_url},)

path = "s3://Pangeo/pangeo-forge/noaa_oisst/v2.1-avhrr.zarr"
ds = xr.open_zarr(fs_osn.get_mapper(path), consolidated=True)
print(f"{ds.nbytes/1e9:.2f} GB")
ds

241.07 GB

,Array,Chunk
Bytes,56.13 GiB,79.10 MiB
Shape,"(14532, 1, 720, 1440)","(20, 1, 720, 1440)"
Count,728 Tasks,727 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,56.13 GiB,79.10 MiB
Shape,"(14532, 1, 720, 1440)","(20, 1, 720, 1440)"
Count,728 Tasks,727 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


# An unbroken _provenance chain_

From source archive ...

https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/

... through `recipe` ...

https://github.com/pangeo-forge/staged-recipes/blob/master/recipes/noaa-oisst/recipe.py

... to Zarr dataset.

**s3://Pangeo/pangeo-forge/noaa_oisst/v2.1-avhrr.zarr**

## Today
- Start working on a recipe: https://pangeo-forge.readthedocs.io/en/latest/
- Add it to the queue for automated builds: https://github.com/pangeo-forge/staged-recipes/pulls
- Build it using a notebook

## Soon
- Build your recipe in an automated "Bakery"
- Browse (and contribute to) a STAC catalog of available Zarr datasets